## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


<li>Похоже тем что используют и туже матрицу(транпонированную в одном из случаев), используют для поиска KNN
<li>Да можно, транспонировав матрицу
<li>item-item использует KNN, a ALS использует градиентный спуск, минимизируя функцию потерь создает две матрицы UxK и KxI, чтобы потом их перемножить и получить предсказания по всей матрице

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


цена

прибыль

срок годности

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


АЛС учитывает  линейные зависимости

## Практическая часть


In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


import os, sys


In [58]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

def get_recommendations(user, model, N=5):
    result = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],
                                    #filter_items=None, 
                                    recalculate_user=True)]
    return result

In [89]:
data = pd.read_csv('retail_train.csv')
data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0


In [60]:
#data.columns = [col.lower() for col in data.columns]
#data.rename(columns={'household_key': 'user_id','product_id': 'item_id'},inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [62]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(5)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [63]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [64]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

sparse_user_item = csr_matrix(user_item_matrix).tocsr() # переведем в формат saprse matrix

sparse_user_item

In [ ]:
user_item_matrix.head(5)

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
matrix_userids,matrix_itemids

In [ ]:
model = AlternatingLeastSquares(factors=100, regularization=0.001, iterations=15, calculate_training_loss=True, num_threads=8) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


In [ ]:
result['count'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='mean', 
                                  fill_value=0 )


user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit
sparse_user_item = csr_matrix(user_item_matrix).tocsr() # переведем в формат saprse matrix

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

model = AlternatingLeastSquares(factors=100, regularization=0.001, iterations=15, calculate_training_loss=True, num_threads=8) # K - кол-во билжайших соседей
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result['mean'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

def get_recommendations(user, model, N=5):
    result = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],
                                    #filter_items=None, 
                                    recalculate_user=True)]
    return result

In [ ]:
result

In [ ]:
df = pd.DataFrame(columns=['aggfunc', 'presision_at_k'])
i = 0
for column in result.columns[2:]:
    presision = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 5)
    df.loc[i] = [column, presision]
    i += 1

df.sort_values(by='presision_at_k', ascending=False)

In [65]:
def matrix_weight_test(data, functioname):
    global sparse_user_item
    global userid_to_id
    global itemid_to_id
    global id_to_itemid
    user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc= functioname, 
                                  fill_value=0 )
    
    user_item_matrix = user_item_matrix.astype(float) 
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()
    
    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values
    
    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))
    
    id_to_itemid = dict(zip(matrix_itemids, itemids))
    id_to_userid = dict(zip(matrix_userids, userids))
    
    itemid_to_id = dict(zip(itemids, matrix_itemids))
    userid_to_id = dict(zip(userids, matrix_userids))
    
    model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.001,
                                    iterations=10, 
                                    calculate_training_loss=True, 
                                    num_threads=0) 
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=False)
    result[functioname] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [66]:
%%time
list = ['mean','nunique','max','size','count']
for aggfunc in list:
    matrix_weight_test(data_train, aggfunc)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Wall time: 7min 16s


In [ ]:
!set MKL_NUM_THREADS=1

In [67]:
df = pd.DataFrame(columns=['aggfunc', 'presision_at_k'])
i = 0
for column in result.columns[2:]:
    presision = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 5)
    df.loc[i] = [column, presision]
    i += 1

df.sort_values(by='presision_at_k', ascending=False)

,aggfunc,presision_at_k
1,nunique,0.20411
0,mean,0.19119
2,max,0.18854
3,size,0.18071
4,count,0.17591


Наилучший результат у **nunique** 

### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [71]:
user_item_matrix = pd.pivot_table(data_train, 
                              index='user_id', columns='item_id', 
                              values='quantity', 
                              aggfunc='nunique', # Лучший параметр 
                              fill_value=0
                             )
user_item_matrix = user_item_matrix.astype(float) 
sparse_user_item = csr_matrix(user_item_matrix).tocsr()
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values
matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))
id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))
itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [72]:
valid = data_train.groupby('user_id')['item_id'].unique().reset_index()
valid.columns=['user_id', 'actual']

In [74]:
%%time

result_k_valid = dict()
n=1

for factors in range(20,220,40):
    for regularization in np.geomspace(0.0001,0.1,4):

        model = AlternatingLeastSquares(factors=int(factors), 
                                        regularization=regularization,
                                        iterations=8, 
                                        calculate_training_loss=True, 
                                        num_threads=0)

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),show_progress=False)
        
        # train data
        result[f'fs={factors}_rg={regularization}'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

        k = result.apply(lambda row: precision_at_k(row[f'fs={factors}_rg={regularization}'], row['actual']), axis=1).mean()
        # test data
        valid[f'fs={factors}_rg={regularization}'] = valid['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

        k_valid = valid.apply(lambda row: precision_at_k(row[f'fs={factors}_rg={regularization}'],row['actual']), axis=1).mean()
        
        result_k_valid[n] = [factors, regularization, k, k_valid]
        n+=1

Wall time: 42min 48s


In [75]:
summary = pd.DataFrame.from_dict(result_k_valid,orient='index',columns=['factors', 'regularization',"train_precision_at_k", "valid_precision_at_k"])
summary[['factors', 'regularization',"train_precision_at_k", "valid_precision_at_k"]].sort_values(by='valid_precision_at_k', ascending=False).head(10)

,factors,regularization,train_precision_at_k,valid_precision_at_k
18,180,0.0010,0.178648,0.948219
17,180,0.0001,0.178355,0.947899
19,180,0.0100,0.182566,0.945418
20,180,0.1000,0.178648,0.943577
14,140,0.0010,0.191969,0.930132
13,140,0.0001,0.187071,0.928211
15,140,0.0100,0.192458,0.924290
16,140,0.1000,0.193732,0.921569
10,100,0.0010,0.205191,0.896999
11,100,0.0100,0.201861,0.894038


In [21]:
#compression_opts = dict(method='zip', archive_name='out.csv')  
summary.to_csv('out.csv', index=False)  

Слишком высокий precision_at_k на тестовых данных. Может правильнее считать наоборот **precision_at_k**(row['actual'],row[f'fs={factors}_rg={regularization}']):

In [91]:
valid.apply(lambda row: precision_at_k(row['actual'],row['fs=180_rg=0.0001']), axis=1).mean()

0.06017740429505144